In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from timeit import default_timer as timer
from scipy.integrate import simps
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from gekko import GEKKO
from MSEIR import MSEIR



In [14]:
city_args = {'n_periods': 200, 'N': 100000,
               'R_0': 2.0, 'tLat': 2, 'tInf': 2, 'tHosp': 2,
               'pMild': 0.90, 'pFatal': 0.20, 'pOvf':0.20,
               'Q': 200, 'S0': 100000-1,
               'E0': 0, 'I0': 1, 'H0': 0, 'R0': 0, 'D0': 0}

### Gekko model

In [15]:

tLat = city_args['tLat']
tInf = city_args['tInf']
tHosp = city_args['tHosp']

pMild = city_args['pMild']
pFatal= city_args['pFatal']

R0 = city_args['R_0']
N = city_args['N']

beta = R0/tInf

# fraction of infected and recovered individuals
e_initial = city_args['E0']
i_initial = city_args['I0']
h_initial = city_args['H0']
r_initial = city_args['R0']
d_initial = city_args['D0']
s_initial = city_args['S0']

m = GEKKO()
u = m.MV(0,lb=0.0,ub=0.8)

s,e,i,h,r,d = m.Array(m.Var,6)

s.value = s_initial
e.value = e_initial
i.value = i_initial
h.value = h_initial
r.value = r_initial
d.value = d_initial

m.Equations([s.dt()== -(1 - u)*beta*s*i/N,\
             e.dt()== (1 - u)*beta*s*i/N - e/tLat,\
             i.dt()== e/tLat - i/tInf,\
             h.dt()==(1-pMild)*i/tInf - h/tHosp,\
             r.dt()== pMild*i/tInf + (1-pFatal)*h/tHosp,\
             d.dt()== pFatal*h/tHosp])

t = np.linspace(0, city_args['n_periods'], int(city_args['n_periods']/2+1))
#t = np.insert(t,1,[0.001,0.002,0.004,0.008,0.02,0.04,0.08, 0.2,0.4,0.8])

m.time = t

# initialize with simulation
m.options.IMODE=7
m.options.NODES=3
m.solve(disp=False)


ImportError: No solution or server unreachable.
  Show errors with m.solve(disp=True).
  Try local solve with m=GEKKO(remote=False).

In [ ]:
max(h.value)

In [ ]:
# optimize
m.options.IMODE=6
m.options.MAX_ITER = 2000
h.UPPER = city_args['Q']
u.STATUS = 1
m.options.SOLVER = 3
m.options.TIME_SHIFT = 1
s.value = s.value.value
e.value = e.value.value
i.value = i.value.value
h.value = h.value.value
r.value = r.value.value
d.value = d.value.value
m.Minimize(u)
m.solve(disp=True)

In [ ]:
# plot the optimized response
plt.figure(figsize=(16,10))
plt.subplot(3,1,1)
plt.plot(m.time, s.value, color='blue', lw=3, ls='--', label='Optimal Susceptible')
plt.plot(m.time, r.value, color='red',  lw=3, ls='--', label='Optimal Recovered')
plt.plot(m.time, d.value, color='black',  lw=3, ls='--', label='Optimal Deceased')
plt.ylabel('Fraction')
plt.legend()

plt.subplot(3,1,2)
plt.plot(m.time, e.value, color='purple', ls='--', lw=3, label='Optimal Exposed')
plt.plot(m.time, i.value, color='orange', ls='--', lw=3, label='Optimal Infected')
plt.plot(m.time, h.value, color='dodgerblue', ls='--', lw=3, label='Optimal Hosp')
plt.ylabel('Fraction')
plt.legend()

plt.subplot(3,1,3)
plt.plot(m.time, u.value, 'k:', lw=3, label='Optimal (0=None, 1=No Interaction)')
plt.ylabel('Social Distancing')
plt.legend()

plt.xlabel('Time (days)')

plt.show()

### MSEIR model

In [ ]:
start = timer()

model = MSEIR(**city_args)
mseirRES = model.solve(U=0, optimize=True, solver='SLSQP', freq=1, hor=2.5, bounds=(0,0.8))[::20]
fig0 = model.plot(mseirRES, comps='HD', size=(700,900), title='Model comparison: MSEIR vs Gekko+IPOPT')

print(timer() - start)

### Comparison

In [ ]:

res_np = np.asarray([s,e,i,h,r,d,t,u,np.zeros(len(t)),np.zeros(len(t)),city_args['Q'] * np.ones(len(t))]).T
df_names = ['S', 'E', 'I', 'H', 'R', 'D', 't', 'Uf', 'mInf', 'rInf', 'Q']
gekkoRES = pd.DataFrame(res_np, columns=df_names)
fig1 = model.plot(gekkoRES, comps='HD')


In [ ]:
simps(mseirRES['Uf']) - simps(gekkoRES['Uf'])


In [ ]:
fig1['data'][1]['line']['color']='dodgerblue'
fig1['data'][3]['line']['color']='dodgerblue'
fig1['data'][2]['line']['color']='dodgerblue'


figF = make_subplots(rows=2, cols=1, shared_xaxes=True,
                            horizontal_spacing=0.01,
                            vertical_spacing=0.01,
                            row_heights=[0.2, 0.8],
                    )

figF.add_trace(fig0['data'][1], row=1, col=1)
figF.add_trace(fig0['data'][3], row=2, col=1)
figF.add_trace(fig0['data'][2], row=2, col=1)


figF.add_trace(fig1['data'][1], row=1, col=1)
figF.add_trace(fig1['data'][3], row=2, col=1)
figF.add_trace(fig1['data'][2], row=2, col=1)


figF

figF.update_yaxes(title_text="Control", row=1, col=1, nticks=4, showgrid=False)
figF.update_yaxes(title_text="Compartments", row=2, col=1, nticks=4,showgrid=False)
        
figF.update_layout(height=800, width=900,
                   title='Model comparison: MSEIR vs Gekko+IPOPT',
                   legend_orientation="h",
                   legend={'bgcolor': 'rgba(0,0,0,0)', 'itemsizing': 'constant'},
                   title_x=0.45, title_y=0.93)


### Summary

In [ ]:

xT = pd.date_range(start = '2020-01-01',
                   end = datetime.strptime('2020-01-01', "%Y-%m-%d")+timedelta(days=200),
                   periods=730)

maxI1 = (mseirRES['D'].max() + mseirRES['R'].max())
maxI2 = (gekkoRES['D'].max() + gekkoRES['R'].max())

maxD1 = mseirRES['D'].max().round(0)
maxD2 = gekkoRES['D'].max().round(0)

perD1 = mseirRES['D'].max() / (mseirRES['D'].max() + mseirRES['R'].max())
perD2 = gekkoRES['D'].max() / (gekkoRES['D'].max() + gekkoRES['R'].max())

perI1 = maxI1/mseirRES['S'].max()
perI2 = maxI2/gekkoRES['S'].max()

idx = mseirRES['Uf'].to_numpy().nonzero()[0]
i = mseirRES['H'] - mseirRES['Q']
min_U1 = xT[idx[-1] if idx.size > 0 else 0]
min_Ul1 = idx.size
area_U1 = round(simps(mseirRES['Uf']),2)
cost_U1 = round(simps((i+abs(i))/2) + simps(mseirRES['Uf']),2)

idx = gekkoRES['Uf'].to_numpy().nonzero()[0]
i = gekkoRES['H'] - gekkoRES['Q']
min_U2 = xT[idx[-1] if idx.size > 0 else 0].date()
min_Ul2 = idx.size
area_U2 = round(simps(gekkoRES['Uf']),2)
cost_U2 = round(simps((i+abs(i))/2) + simps(gekkoRES['Uf']),2)

total__ = [[area_U1, min_U1, min_Ul1, maxI1, perI1, maxD1, perD1, cost_U1],
           [area_U2, min_U2, min_Ul2, maxI2, perI2, maxD2, perD2, cost_U2]]

cols = ['Control strength', 'Control release date', 'Control duration',
        'Total Infected', 'Total Infected (% population)', 'Total Deceased',
       'Total Deceased (% infected)', 'Final value of cost function']

table_summary = pd.DataFrame(total__, columns=cols)
table_summary.index = ['Scenario 1', 'Scenario 2']
table_summary.T

